In [1]:
from IPython.display import Image

- https://www.bilibili.com/video/BV1TMRHYJEaw/

## qwen-vl

In [2]:
Image(url='./imgs/qwen-vl-pipeline.png', width=400)

- Visual Encoder
    - pre-trained weights from Openclip’s ViT-bigG
- (Position-aware Vision-Language) Adapter
    - single-layer cross-attention module
        - a group of trainable vectors (Embeddings) as **query** vectors
        - image features from the visual encoder as **keys** for crossattention operations
        - 2D absolute positional encodings
- inputs/outputs
    - `<img></img>`
    - `<box></box>`

### Data Format of Multi-Task Pre-training

In [3]:
Image(url='./imgs/qwen-vl-pretrain.png', width=600)

### Supervised Fine-tuning

In [4]:
Image(url='./imgs/qwen-vl-sft.png', width=600)

## qwen2.5-vl

### naive dynamic resolution (原生动态分辨率)

-  (1092, 8204)
    - 在训练和推理过程中，输入图像的高度和宽度在送入ViT之前会被调整为28的倍数。
        - 宽度 1092 / 28 = 39 (已经是28的倍数)
        - 高度 8204 / 28 = 293 (已经是28的倍数)
    - 图像切块 (Patching)：
        - 视觉编码器通过将图像分割成 **步长（stride）为14的块（patches）** 来处理图像，生成一组图像特征。
            - 我们使用 14×14 的图像块作为基本单元”。
            - 这意味着图像会被切分成 14x14 大小的、不重叠的块。
        - 在宽度方向上的块数量：1092 / 14 = 78 个块；在高度方向上的块数量：8204 / 14 = 586 个块
            - 所以，ViT最初生成的原始图像特征（或称为“视觉token”）数量是：78 * 586 = 45708 个原始视觉token。
    - ViT Hidden Size
        - "Vision Transformer (ViT)" 部分的 "Hidden Size" 为 1280。
    - Vision-Language Merger
        - 我们首先将空间上相邻的四个图像块特征进行分组。这些分组后的特征随后被连接并通过一个两层的多层感知器（MLP）投影到一个与LLM中使用的文本嵌入对齐的维度。这种方法不仅降低了计算成本，还提供了一种灵活的方式来动态压缩不同长度的图像特征序列。
            - 45708/4 = 11427
        - 4 × 1280 = 5120维 => 2层 MLP
            - Qwen2.5-VL-72B模型，LLM的隐藏层维度是 8192 (根据表1 LLM Hidden Size)。

### 动态 fps 采样训练